### Summary

This is a practice to compute item `unit_price` given item purchase `quantity`, `item_code`, `total invoice price`. The key idea is to pivot the table such that `item_code` becomes the columns with quantity in each invoice. If we have sufficient number of invoices, then it can be solved using linear equation system with `p` variables and `n` equations, where `p` represents the number of unique items and `n` represent the number of invoices.

In [34]:
import numpy as np
import pandas as pd
from random import sample

The following function defines a method to get a non-singular matrix from the pivoting data so that we can get a robust solution. It is possible there are errors due to insufficient observations, but the function can be run several times so that a valid non-singular matrix is drawn, thus returning one solution to the system of linear equations. If the condition of sufficiently large observations is satisfied, the solution will be returned and will be only one.

The errors returned include __Singular Matrix__, __Sample larger than population or is negative__, which means the square matrix we sampled does not meet the requirements to solve the system of linear equations due to insufficient observation. We can simply run the function several times until we get the solution.

In [35]:
def get_UnitPrice(item_group):
    # get number of invoice and number of items
    n = item_group.shape[0]
    p = item_group.shape[1]-1
    
    # compute the rank of the data matrix
    mtx_rank = np.linalg.matrix_rank(item_group.iloc[:, 0:p].to_numpy())
    if n < p or mtx_rank != p:
        print('There is no solution to get the unit price for each item!')
    
    # initialize the indexes that are going to be extracted
    idx = np.zeros(p)
    idx[0] = sample(list(item_group.index[(item_group.iloc[:, 0] != 0)]), 1)[0]
    
    # initialize the square matrix for solving linear equations
    X = item_group.loc[idx[0], :]
    
    # extract one obserevation for each item_code with quantity greater than 0
    for i in range(1, p):
        # get indexes with quantity greater than 0
        idx_not_zero = (item_group.iloc[:, i] != 0)
        
        # get indexes that have been not extracted from all previous item_code
        idx_not_in = ~item_group.index.isin(idx[0:i])
        
        # get the list of indexes from the grouped data
        idx_list = list(item_group.index[idx_not_zero & idx_not_in])
        
        # only sample one from the list of indexes 
        idx_sample = sample(idx_list, 1)[0]
        
        # store in the idx array
        idx[i] = idx_sample
        
        # concatenate new observation
        X = pd.concat([X, item_group.loc[idx[i], :]], axis=1)
    # transpose the dataframe to have item_code as columns
    X = X.T
    unit_price = np.linalg.solve(X.to_numpy()[:, 0:-1], X.to_numpy()[:, -1])
    return(unit_price)

### Apple Invoice

Let's first look at an example of apple products invoice:

- unit_price = {'iPhone 13 Pro': 999, 'Macbook Pro': 1499, 'AirPods': 179}
- 1001: 999 * 2 + 1499 * 1 = 3497
- 1002: 1499 * 2 + 179 * 3 + 999 * 4 = 7531
- 1003: 179 * 7 = 7531
- 1004: 1499 * 3 = 4497

Given the above setting, simulated data is generated

In [36]:
raw_data = {'invoice_id': [1001, 1001, 1002, 1002, 1002, 1003, 1004],
            'item_code': ['iPhone 13 Pro', 'Macbook Pro', 'Macbook Pro', 'AirPods', 'iPhone 13 Pro', 'AirPods', 'Macbook Pro'],
            'quantity': [2, 1, 2, 3, 4, 7, 3],
            'total_price': [3497, 3497, 7531, 7531, 7531, 1253, 4497]}
invoice = pd.DataFrame(raw_data)
invoice

,invoice_id,item_code,quantity,total_price
0,1001,iPhone 13 Pro,2,3497
1,1001,Macbook Pro,1,3497
2,1002,Macbook Pro,2,7531
3,1002,AirPods,3,7531
4,1002,iPhone 13 Pro,4,7531
5,1003,AirPods,7,1253
6,1004,Macbook Pro,3,4497


In [37]:
# pivot the table with item_code as column names and sum the quantity
item_group = pd.pivot_table(data=invoice, 
                        index=['invoice_id'], 
                        columns=['item_code'], 
                        values=['quantity'],
                        aggfunc='sum').fillna(0)
# drop column level: quantity
item_group.columns = item_group.columns.droplevel()

# merge with all the unique total price
Y = invoice.groupby('invoice_id').total_price.mean()
item_group = pd.merge(item_group, Y, on='invoice_id')

# view item_group
item_group

,AirPods,Macbook Pro,iPhone 13 Pro,total_price
invoice_id,,,,
1001,0.0,1.0,2.0,3497.0
1002,3.0,2.0,4.0,7531.0
1003,7.0,0.0,0.0,1253.0
1004,0.0,3.0,0.0,4497.0


In [38]:
unit_price = get_UnitPrice(item_group)
print(unit_price)

[ 179. 1499.  999.]


### Simulated Invoice

The following example is a simulated data of invoice with 50 unique items, 70 invoices and quantites ranging between 1 and 100. Since 70 is a relatively smaller number of observations compared to 50 unique items, it is possible that the invoice breakdown did not cover all the products. To efficiently get a valid solution, more invoices need to be collected for analysis.

In [107]:
import random
import string
num_item = 50
s1 = pd.Series(random.sample(list(string.ascii_letters), num_item), name='item_code')
s2 = pd.Series(np.random.randint(low=1, high=100, size=num_item), name='quantity')
s3 = pd.Series(random.sample(range(1, 1500), num_item), name='unit_price').astype(float)
df = pd.concat([s1, s2, s3], axis=1)
df

,item_code,quantity,unit_price
0,O,13,723.0
1,p,11,371.0
2,m,89,694.0
3,Z,3,675.0
4,S,29,706.0
5,c,89,246.0
6,F,89,891.0
7,C,80,1248.0
8,D,44,261.0
9,H,73,1004.0


In [108]:
invoice = pd.DataFrame()
num_invoice = 100
for i in range(num_invoice):
    # Step 1: randomly sample an integer to represent the number of unique items in one invoice
    m = int(np.random.randint(low=1, high=8, size=1))
    
    # Step 2: randomly sample m unique item_code, get the index and correpondent dataframe
    idx = random.sample(range(num_item), m)
    X = df.iloc[idx, :].copy()

    # Step 3: compute the total price by multuplying unit_price with quantity
    total_price = (X.quantity * X.unit_price).sum()

    # Step 4: append total_price to the dataframe and add a new column named invoice_id
    X['total_price'] = total_price
    X['invoice_id'] = 1000+i
    invoice = invoice.append(X)

# delete the unit price and reset index
invoice = invoice.drop('unit_price', axis=1).reset_index(drop=True)
invoice.head()

,item_code,quantity,total_price,invoice_id
0,P,31,178645.0,1000
1,f,61,178645.0,1000
2,W,66,178645.0,1000
3,e,34,231516.0,1001
4,m,89,231516.0,1001


In [109]:
# pivot the table with item_code as column names and sum the quantity
item_group = pd.pivot_table(data=invoice, 
                        index=['invoice_id'], 
                        columns=['item_code'], 
                        values=['quantity'],
                        aggfunc='sum').fillna(0)
# drop column level: quantity
item_group.columns = item_group.columns.droplevel()

# merge with all the unique total price
Y = invoice.groupby('invoice_id').total_price.mean()
item_group = pd.merge(item_group, Y, on='invoice_id')

# view item_group
item_group.head()

,A,B,C,D,E,F,G,H,I,J,...,q,r,t,u,v,w,x,y,z,total_price
invoice_id,,,,,,,,,,,,,,,,,,,,,
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,178645.0
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,231516.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69740.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0,108506.0
1004,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0,50323.0


In [110]:
unit_price = get_UnitPrice(item_group)
print(unit_price)

[ 447. 1182. 1248.  261.  992.  891.  800. 1004. 1366.   96. 1381.  958.
  795. 1153.  723.  696.   90.  729.  706. 1077.  197.  973. 1475.  557.
  595.  675. 1465.  604.  246.  743. 1379.  979. 1083.  370. 1472.  356.
  187.  753.  694.  771.  371. 1240. 1159.  903.  182.  741.  284.  355.
 1185.  843.]


In [111]:
# check if match the preset unit_price
sol = pd.Series(unit_price).sort_values().reset_index(drop=True)
org = df.unit_price.sort_values(ascending=True).reset_index(drop=True)
tol = 0.1
if (sol - org).sum() < tol:
    print('The solution is correct.')
    print(sol.values)
else:
    print('There is no robust solution.')

The solution is correct.
[  90.   96.  182.  187.  197.  246.  261.  284.  355.  356.  370.  371.
  447.  557.  595.  604.  675.  694.  696.  706.  723.  729.  741.  743.
  753.  771.  795.  800.  843.  891.  903.  958.  973.  979.  992. 1004.
 1077. 1083. 1153. 1159. 1182. 1185. 1240. 1248. 1366. 1379. 1381. 1465.
 1472. 1475.]
